In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy

In [18]:
import cv2 as cv
import os
import numpy as np

In [19]:
DENUMIRE_FOLDER_TEST = 'antrenare'
DENUMIRE_FOLDER_IMAGINI_PROPRII = 'imagini_proprii'
DENUMIRE_FOLDER_ADNOTARI = '334_Florete_FabianAndrei'

In [20]:
class Cell():
    def __init__(self):
        self.multiplier = 1
        self.value = -1
        self.constraint = None

class Board():
    def __init__(self):
        self.matrix = [[Cell() for _ in range(14)] for j in range(14)]
        self.available_pieces = {0:1, 1:7, 2:7, 3:7, 4:7, 5:7, 6:7, 7:7, 8:7, 9:7, 10:7,
                                11:1, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1,
                                20:1, 21:1, 24:1, 25:1, 27:1, 28:1,
                                30:1, 32:1, 35:1, 36:1,
                                40:1, 42:1, 45:1, 48:1, 49:1, 
                                50:1, 54:1, 56:1, 60:1, 63:1, 64:1,
                                70:1, 72:1, 80:1, 81:1, 90:1}
        self.matrix[0][0].multiplier = 3
        self.matrix[0][6].multiplier = 3
        self.matrix[0][7].multiplier = 3
        self.matrix[0][13].multiplier = 3
        self.matrix[1][1].multiplier = 2
        self.matrix[1][4].constraint = '/'
        self.matrix[1][9].constraint = '/'
        self.matrix[1][12].multiplier = 2
        self.matrix[2][2].multiplier = 2
        self.matrix[2][5].constraint = '-'
        self.matrix[2][8].constraint = '-'
        self.matrix[2][11].multiplier = 2
        self.matrix[3][3].multiplier = 2
        self.matrix[3][6].constraint = '+'
        self.matrix[3][7].constraint = '*'
        self.matrix[3][10].multiplier = 2
        self.matrix[4][1].constraint= '/'
        self.matrix[4][4].multiplier = 2
        self.matrix[4][6].constraint = '*'
        self.matrix[4][7].constraint = '+'
        self.matrix[4][9].multiplier = 2
        self.matrix[4][12].constraint = '/'
        self.matrix[5][2].constraint = '-'
        self.matrix[5][11].constraint = '-'
        self.matrix[6][0].multiplier = 3
        self.matrix[6][3].constraint = '*'
        self.matrix[6][4].constraint = '+'
        self.matrix[6][6].value = 1
        self.matrix[6][7].value = 2
        self.matrix[6][9].constraint = '*'
        self.matrix[6][10].constraint = '+'
        self.matrix[6][13].multiplier = 3
        self.matrix[7][0].multiplier = 3
        self.matrix[7][3].constraint = '+'
        self.matrix[7][4].constraint = '*'
        self.matrix[7][6].value = 3
        self.matrix[7][7].value = 4
        self.matrix[7][9].constraint = '+'
        self.matrix[7][10].constraint = '*'
        self.matrix[7][13].multiplier = 3
        self.matrix[8][2].constraint = '-'
        self.matrix[8][11].constraint = '-'
        self.matrix[9][1].constraint = '/'
        self.matrix[9][4].multiplier = 2
        self.matrix[9][6].constraint = '+'
        self.matrix[9][7].constraint = '*'
        self.matrix[9][9].multiplier = 2
        self.matrix[9][12].constraint = '/'
        self.matrix[10][3].multiplier = 2
        self.matrix[10][6].constraint = '*'
        self.matrix[10][7].constraint = '+'
        self.matrix[10][10].multiplier = 2
        self.matrix[11][2].multiplier = 2
        self.matrix[11][5].constraint = '-'
        self.matrix[11][8].constraint = '-'
        self.matrix[11][11].multiplier = 2
        self.matrix[12][1].multiplier = 2
        self.matrix[12][4].constraint = '/'
        self.matrix[12][9].constraint = '/'
        self.matrix[12][12].multiplier = 2
        self.matrix[13][0].multiplier = 3
        self.matrix[13][6].multiplier = 3
        self.matrix[13][7].multiplier = 3
        self.matrix[13][13].multiplier = 3
        

    def get_piece(self, i, j):
        return self.matrix[i][j].value

    def set_piece(self, i, j, val):
        self.matrix[i][j].value = val
        self.available_pieces[val] = self.available_pieces[val] - 1

In [21]:
DIMENSIUNE_PATRAT = 140
HEIGHT_CAREU = 14 * DIMENSIUNE_PATRAT
WIDTH_CAREU = 14 * DIMENSIUNE_PATRAT

lines_vertical = []
for i in range(0, WIDTH_CAREU+5, DIMENSIUNE_PATRAT):
    l = []
    l.append((i, 0))
    l.append((i, WIDTH_CAREU-3))
    lines_vertical.append(l)

lines_horizontal = []
for i in range(0, HEIGHT_CAREU+5, DIMENSIUNE_PATRAT):
    l = []
    l.append((0, i))
    l.append((HEIGHT_CAREU - 3, i))
    lines_horizontal.append(l)

In [22]:
folder_proiect = os.getcwd() #Tema1
folder_imagini_proprii = os.path.join(folder_proiect, DENUMIRE_FOLDER_IMAGINI_PROPRII)
folder_imagini = os.path.join(folder_proiect, DENUMIRE_FOLDER_TEST) # Tema1/antrenare, aici se va schimba in testare
continut_folder_imagini = [x for x in os.listdir(folder_imagini) if x.endswith('.txt') or x.endswith('.jpg')]

if DENUMIRE_FOLDER_ADNOTARI not in os.listdir():
    os.makedirs(DENUMIRE_FOLDER_ADNOTARI)

def show_image(title, image):
    image = cv.resize(image,(0,0), fx=0.1, fy=0.1)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def read_image(image_path):
    #openCv o citeste ca fiind BGR
    img = cv.imread(image_path)
    return img.copy()

def read_turns(id_imagine, unraveled = True):
    f = open(os.path.join(folder_imagini, f'{id_imagine}_turns.txt'))
    turns = []
    unraveled_turns = []
    for line in f.readlines():
        text = line.split()
        turns.append((text[0], int(text[1])))
    f.close()

    if unraveled == False:
        return turns
    
    for i in range(len(turns) - 1):
        n = turns[i + 1][1] - turns[i][1]
        #print(turns[i + 1][1], turns[i][1], turns[i + 1][1] - turns[i][1])
        player = turns[i][0]
        unraveled_turns.extend([player for _ in range(n)])

    unraveled_turns.extend([turns[-1][0] for _ in range(51 - turns[-1][1])])
    return unraveled_turns
    
def apply_hsv_filter(image, l_h, l_s, l_v, u_h, u_s, u_v, verbose = False):
    #BGR in BGR out
    #h -> hue
    #s -> saturation
    #v -> value
    #l -> low
    #h -> high
    copie = image.copy()
    copie = cv.cvtColor(copie, cv.COLOR_BGR2HSV)
    l = np.array([l_h, l_s, l_v])#lower bound pe cele 3 canale
    u = np.array([u_h, u_s, u_v])#upper bound pe cele 3 canale
    mask_table_hsv = cv.inRange(copie, l, u)
    
    if verbose:
        show_image('masca_hsv', mask_table_hsv)
        
    rez = cv.bitwise_and(copie, copie, mask=mask_table_hsv)
    rez = cv.cvtColor(rez, cv.COLOR_HSV2BGR)
    return rez

def _find_contour_positions(edges, verbose = False):

    if verbose:
        show_image('edge_image', edges)
        
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    return top_left, top_right, bottom_left, bottom_right

def extrage_tabla_joc(imagine, verbose = False):
    #in IMAGINE out TABLA DE JOC in BGR(Cu tot cu scrisul Mathable)
    original = imagine.copy()

    if verbose:
        show_image('imagine_originala', original)
    
    imagine = apply_hsv_filter(imagine, 14, 0, 0, 255, 255, 255)#primul parametru e bun de la 14 in sus

    if verbose:
        show_image('imagine_dupa_filtru_hsv', imagine)

    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(imagine, 5)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 18)
    # image_sharpened = cv.addWeighted(image_m_blur, -0.8, image_g_blur, 1.2, 0)
    # image_sharpened = image_m_blur
    # show_image('imagine_dupa_sharpening', image_m_blur)
    
    # _, thresh = cv.threshold(image_sharpened, 40, 255, cv.THRESH_BINARY)
    # kernel = np.ones((6,6), np.uint8)
    # thresh = cv.erode(thresh, kernel)
    # show_image('image_thresholded', thresh)

    # edges = cv.Canny(thresh, 10, 255)

    top_left, top_right, bottom_left, bottom_right = _find_contour_positions(imagine, verbose)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    if verbose:
        show_image("detected corners",image_copy)

    width = top_right[0] - top_left[0] #18x18, each cell will have 80 pixels
    height = width
    tabla_joc = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    colutri = np.array([[0,0], [width, 0], [width, height], [0,height]], dtype='float32')
    M = cv.getPerspectiveTransform(tabla_joc, colutri)
    result = cv.warpPerspective(original, M, (width, height))
    crop_image_pixels = 85
    return result[crop_image_pixels:result.shape[0] - crop_image_pixels, 
                    crop_image_pixels:result.shape[1] - crop_image_pixels]



In [23]:
#edge detection is susceptible to noise in the image

In [24]:
#ATENTIE LA CELULA ASTA https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
def _extrage_careu_tabla(edges, imagine, verbose = False):
    #show_image('edge_image', edges)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    squares = []
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            squeezed_contours = contours[i].squeeze()
            x_min = np.min(squeezed_contours[:, 0])
            x_max = np.max(squeezed_contours[:, 0])
            y_min = np.min(squeezed_contours[:, 1])
            y_max = np.max(squeezed_contours[:, 1])
            top_left_point = [x_min, y_min]
            top_right_point = [x_max, y_min]
            bottom_left_point = [x_min, y_max]
            bottom_right_point = [x_max, y_max]
            height = y_max - y_min
            width = x_max - x_min
            diagonal = np.sqrt(height**2 + width**2)

            if diagonal > 50:#find a good diagonal threshold
                cv.circle(imagine, tuple(top_left_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(top_right_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(bottom_left_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(bottom_right_point), 20, (0,0, 255), -1)
                squares.append([top_left_point, top_right_point, bottom_left_point, bottom_right_point])

    squares_np = np.array(squares)

    if verbose:
        show_image('detected corners', imagine)

    x_min = np.min(squares_np[:, :, 0])
    x_max = np.max(squares_np[:, :, 0])
    y_min = np.min(squares_np[:, :, 1])
    y_max = np.max(squares_np[:, :, 1])
    top_left_point = [x_min, y_min]
    top_right_point = [x_max, y_min]
    bottom_left_point = [x_min, y_max]
    bottom_right_point = [x_max, y_max]
    
    return (top_left_point, top_right_point, bottom_left_point, bottom_right_point)

def extrage_careu_joc(imagine, verbose = False):
    #in TABLA DE JOC out DOAR CAREUL in BGR(cred)
    original = imagine.copy()
    
    if verbose:
        show_image('imagine_tabla_joc', imagine)

    copie = imagine.copy()
    
    imagine = apply_hsv_filter(imagine, 14, 0, 0, 69, 255, 255)
    
    if verbose:
        show_image('imagine_dupa_hsv', imagine)
    
    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(imagine, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 5)
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    
    if verbose:
        show_image('image_sharpened', image_sharpened)
    
    _, thresh = cv.threshold(image_sharpened, 0, 255, cv.THRESH_BINARY)

    kernel = np.ones((10,10), np.uint8)
    thresh = cv.erode(thresh, kernel)
    
    if verbose:
        show_image('image_thresholded', thresh)

    edges = cv.Canny(thresh, 200, 400)
    #top_left, top_right, bottom_left, bottom_right = find_contour_positions(edges)
    top_left, top_right, bottom_left, bottom_right = _extrage_careu_tabla(edges, copie, verbose)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    if verbose:
        show_image("detected corners",image_copy)

    height = HEIGHT_CAREU
    width = WIDTH_CAREU
    
    careuri = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    destinatie_careuri = np.array([[0,0],[width, 0], [width, height], [0, height]], dtype='float32')

    M = cv.getPerspectiveTransform(careuri, destinatie_careuri)
    
    result = cv.warpPerspective(original, M, (width, height)) 

    return result

In [25]:
def cauta_pozitie_piesa_noua(imag1, imag2, board, verbose = False):
    #imag1 este imaginea anterioara, iar imag2 este imaginea curenta unde ne uitam dupa piese
    #in IMAG1, IMAG2 in format BGR out diferenta
    #IMAG1 si IMAG2 sunt imagini doar cu patratele de joc
    #imag2 - imag1
    global lines_horizontal
    global lines_vertical
    
    imag1 = cv.cvtColor(imag1, cv.COLOR_BGR2GRAY)
    imag2 = cv.cvtColor(imag2, cv.COLOR_BGR2GRAY)
    diferenta = cv.absdiff(imag1, imag2)
    _, diferenta = cv.threshold(diferenta, 90, 255, cv.THRESH_BINARY)

    kernel = np.ones((3,3), np.uint8)
    diferenta = cv.erode(diferenta, kernel)

    if verbose:
        show_image('diferenta', diferenta)

    matrice_medie = np.zeros((14,14), dtype='float32')

    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0] + 5# + 10
            y_max = lines_vertical[j + 1][1][0] - 5# - 15
            x_min = lines_horizontal[i][0][1] + 5# + 10
            x_max = lines_horizontal[i + 1][1][1] - 5# - 10
            patch = diferenta[x_min:x_max, y_min:y_max].copy()
            medie = np.mean(patch)
            matrice_medie[i,j] = medie


    index_max = np.argmax(matrice_medie)
    i_max, j_max = np.unravel_index(index_max, matrice_medie.shape)
    while board.get_piece(int(i_max), int(j_max)) != - 1:
        matrice_medie[i_max][j_max] = -1
        index_max = np.argmax(matrice_medie)
        i_max, j_max = np.unravel_index(index_max, matrice_medie.shape)
        
    return int(i_max), int(j_max)#numaratoare de la 0


In [26]:
def adnotari_prima_cerinta(id_imagine, verbose = False):
    board = Board()
    adnotari = []
    base_imagine = read_image(os.path.join(folder_imagini_proprii, 'tabla.jpg'))
    base_tabla = extrage_tabla_joc(base_imagine, verbose)
    base_careu = extrage_careu_joc(base_tabla, verbose)

    if verbose:
        show_image('base_image', base_careu)
    
    for i in range(1, 51):
        imagine = read_image(os.path.join(folder_imagini, f'{id_imagine}_{i//10}{i%10}.jpg'))
        imagine_tabla = extrage_tabla_joc(imagine, verbose)
        imagine_careu = extrage_careu_joc(imagine_tabla, verbose)
        i, j = cauta_pozitie_piesa_noua(base_careu, imagine_careu, board, verbose)

        board.set_piece(i, j, 1)
        
        base_careu = imagine_careu
        adnotari.append((i, j))
    return adnotari

In [27]:
def clasifica_numar(patch, board, verbose):
    #if verbose:
    #    show_image('patch', patch)

    all_templates = []
    for i in board.available_pieces.keys():
        #all_templates.append((read_image(os.path.join(folder_imagini_proprii, f'{i}_separat.jpg')), i))
        #all_templates.append((read_image(os.path.join(folder_imagini_proprii, f'{i}_impreuna.jpg')), i))
        all_templates.append((read_image(os.path.join(folder_imagini_proprii, f'{i}.jpg')), i))
    
    patch = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)

    #preprocess the same as the template
    image_m_blur = cv.medianBlur(patch, 3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 5)
    patch = cv.addWeighted(image_m_blur, 1.6, image_g_blur, -0.9, 0)
    _, patch = cv.threshold(patch, 80, 255, cv.THRESH_BINARY)
    
    corr_list = []
    for template, value in all_templates:
        template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
        corr = cv.matchTemplate(patch, template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)
        corr_list.append((corr, value, template))

    corr_list.sort(key = lambda x: -x[0])
    label = corr_list[0][1]
    best_template = corr_list[0][2]
    if corr_list[0][1] >= 0 and corr_list[0][1] <= 9:
        if verbose:
            print(corr_list[0][0],  corr_list[1][0], corr_list[2][0], corr_list[3][0])
        if (corr_list[0][0] >= 0.80 and corr_list[1][0] >= 0.80):
            if corr_list[1][1] >= 10:
                label = corr_list[1][1]
                best_template = corr_list[1][2]
            elif corr_list[2][1] >= 10:
                label = corr_list[2][1]
                best_template = corr_list[2][2]
            if verbose:
                print(corr_list[0][0], corr_list[1][0], corr_list[2][0], corr_list[3][0])
                show_image('imagine', patch)
                show_image('prima_corelatie', corr_list[0][2])
                show_image('a_doua_corelatie', corr_list[1][2])
                show_image('a_treia_corelatie', corr_list[2][2])

    
    if verbose:
        show_image('patch', patch)
        show_image('template_matched', best_template)
    
    return label

In [28]:
def adnotari_a_doua_cerinta(id_imagine, a, verbose=False):
    #a sunt adntarile de la prima cerinta
    board = Board()
    rez = []
    for i in range(len(a)):
        pos = a[i]
        img = read_image(os.path.join(folder_imagini, f'{id_imagine}_{(i + 1)//10}{(i + 1)%10}.jpg'))
        img = extrage_tabla_joc(img)
        img = extrage_careu_joc(img)
        if verbose:
            show_image('img', img)
        y_min = lines_vertical[pos[1]][0][0]#+ 15
        y_max = lines_vertical[pos[1] + 1][1][0]#- 15
        x_min = lines_horizontal[pos[0]][0][1]#+ 15
        x_max = lines_horizontal[pos[0] + 1][1][1]# - 15
        patch = img[x_min:x_max, y_min:y_max].copy()
        
        numar = clasifica_numar(patch, board, verbose)
        rez.append(numar)

    return rez


In [29]:
def calculeaza_scor_piesa(tabla, poz, piesa, verbose = False):
    #tabla este intre [0, 13] pe ox si [0, 13] pe oy
    def generate_possibilities(a, b, constraint):
        pos = []
        big = max(a, b)
        small = min(a, b)
        if constraint == '/' and small != 0 and big != 0 and big % small == 0:
            pos.append(big // small)
        if constraint == '*':
            pos.append(big * small)
        if constraint == '-':
            pos.append(big - small)
        if constraint == '+':
            pos.append(big + small)
        if constraint == None:
            pos.append(big + small)
            pos.append(big * small)
            pos.append(big - small)
            if small != 0 and big != 0 and big % small == 0:
                pos.append(big // small)
        return pos

    round_score = 0
    directions = []
    if poz[0] >= 2:
        directions.append([(-1, 0), (-2, 0)])
    if poz[0] <= 11:
        directions.append([(1, 0), (2, 0)])
    if poz[1] >= 2:
        directions.append([(0, -1), (0, -2)])
    if poz[1] <= 11:
        directions.append([(0, 1), (0, 2)])

    for direction in directions:
        if verbose:
            print(direction, poz)
        a = tabla.matrix[poz[0] + direction[0][0]][poz[1] + direction[0][1]].value
        b = tabla.matrix[poz[0] + direction[1][0]][poz[1] + direction[1][1]].value
        if verbose:
            print('a', poz[0] + direction[0][0], poz[1] + direction[0][1])
            print('b', poz[0] + direction[1][0], poz[1] + direction[1][1])
        constraint = tabla.matrix[poz[0]][poz[1]].constraint
        if a != -1 and b != -1:
            round_score += piesa * tabla.matrix[poz[0]][poz[1]].multiplier if piesa in generate_possibilities(a, b, constraint) else 0
            if verbose:
                print(piesa * tabla.matrix[poz[0]][poz[1]].multiplier if piesa in generate_possibilities(a, b, constraint) else 0)
    return round_score

In [30]:
def adnotari_a_treia_cerinta(id_imagine, adnotari_a, adnotari_b, verbose = False):
    tabla = Board()
    turns = read_turns(id_imagine)
    score_Player1 = 0
    score_Player2 = 0

    rez = []
    for i in range(50):
        player = turns[i]
        poz = adnotari_a[i]
        piesa = adnotari_b[i]
        if player == 'Player1':
            score_Player1 += calculeaza_scor_piesa(tabla, poz, piesa, verbose)
        if player == 'Player2':
            score_Player2 += calculeaza_scor_piesa(tabla, poz, piesa, verbose)
        tabla.matrix[poz[0]][poz[1]].value = piesa
        
        if i == 49 or player != turns[i + 1]:
            if player == 'Player1':
                rez.append(score_Player1)
            if player == 'Player2':
                rez.append(score_Player2)
            score_Player1 = 0
            score_Player2 = 0
    return rez


In [31]:
def scrie_adnotari(id_imagine, adnotari_a, adnotari_b, adnotari_c):
    folder_solutii = os.path.join(folder_proiect, '334_Florete_FabianAndrei')
    alfabet = [chr(x) for x in range(ord('A'), ord('a') + 15)]
    for i in range(1, 51):
        f = open(os.path.join(folder_solutii, f'{id_imagine}_{i//10}{i%10}.txt'), 'w')
        f.write(f'{adnotari_a[i - 1][0] + 1}{alfabet[adnotari_a[i - 1][1]]} {adnotari_b[i - 1]}')
        f.close()

    f = open(os.path.join(folder_solutii, f'{id_imagine}_scores.txt'), 'w')
    turns = read_turns(id_imagine, unraveled=False)
    for turn, adnotare in zip(turns[:-1], adnotari_c[:-1]):
        f.write(f'{turn[0]} {turn[1]} {adnotare}\n')
    f.write(f'{turns[-1][0]} {turns[-1][1]} {adnotari_c[-1]}') # This looks shady, sper sa mearga
    f.close()
    

In [32]:
for i in range(1, 5):
    a = adnotari_prima_cerinta(i)
    b = adnotari_a_doua_cerinta(i, a)
    c = adnotari_a_treia_cerinta(i, a, b)
    scrie_adnotari(i, a, b, c)

In [33]:
# def extrage_template():

#     img_toate_separat = read_image(os.path.join(folder_imagini_proprii, 'piese_toate_separat.jpg'))
#     img_toate_separat = extrage_tabla_joc(img_toate_separat)
#     img_toate_separat = extrage_careu_joc(img_toate_separat)

#     img_toate_impreuna = read_image(os.path.join(folder_imagini_proprii, 'piese_toate_impreuna.jpg'))
#     img_toate_impreuna = extrage_tabla_joc(img_toate_impreuna)
#     img_toate_impreuna = extrage_careu_joc(img_toate_impreuna)
    
#     board = Board()
#     keys = list(board.available_pieces.keys())
#     cnt = 0
#     cnt_1 = 0
#     for i in range(len(lines_horizontal) - 1):
#         for j in range(len(lines_vertical) - 1):
#             y_min = lines_vertical[j][0][0] + 10
#             y_max = lines_vertical[j + 1][1][0] - 10
#             x_min = lines_horizontal[i][0][1] + 10
#             x_max = lines_horizontal[i + 1][1][1] - 10
#             patch = img_toate_separat[x_min:x_max, y_min:y_max].copy()
#             patch_1 = img_toate_impreuna[x_min:x_max, y_min:y_max].copy()
#             if i >= 5 and i <= 10 and j >= 4 and j <= 11:
#                 try:
#                     cv.imwrite(os.path.join(folder_imagini_proprii, f'{keys[cnt_1]}_impreuna.jpg'), patch_1)
#                 except IndexError:
#                     pass
#                 cnt_1 += 1
#             if i % 2 == 0 and j % 2 == 0:
#                 try:
#                     cv.imwrite(os.path.join(folder_imagini_proprii, f'{keys[cnt]}_separat.jpg'), patch)
#                 except IndexError:
#                     pass
#                 cnt += 1
# extrage_template()

In [16]:
# def centreaza_template(template, val):
#     copie = template.copy()
#     #show_image('template', template)
#     template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
#     image_m_blur = cv.medianBlur(template, 3)
#     image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 5)
#     template = cv.addWeighted(image_m_blur, 1.6, image_g_blur, -0.9,0)
#     _, template = cv.threshold(template, 80, 255, cv.THRESH_BINARY)
#     #show_image('template', template)

#     #kernel = np.ones((5,5), np.uint8)
#     #template = cv.erode(template, kernel)

#     #cv.imwrite(os.path.join(folder_imagini_proprii, 'decupata.jpg'), template)

#     linii = []
#     coloane = []
#     for i in range(len(template)):
#         for j in range(len(template[i])):
#             if template[i][j] == 0:
#                 linii.append(i)
#                 break
#     for j in range(len(template)):
#         for i in range(len(template)):
#             if template[i][j] == 0:
#                 coloane.append(j)
#                 break
    
#     x_min = linii[0]
#     x_max = linii[-1]
#     y_min = coloane[0]
#     y_max = coloane[-1]

#     template = template[x_min:x_max, y_min:y_max]
#     cv.circle(copie, (x_min, y_min), 10, (0,0,255), -1)
#     cv.circle(copie, (x_max, y_max), 10, (0,0,255), -1)

#     #show_image('copie', copie)
#     # print(linii[0], linii[-1])
#     # print(coloane[0], coloane[-1])
    
#     #show_image('template', template)

#     cv.imwrite(os.path.join(folder_imagini_proprii, f'{val}.jpg'), template)


# board = Board()
# all_templates = []
# for i in board.available_pieces.keys():
#     try:
#         all_templates.append((read_image(os.path.join(folder_imagini_proprii, f'{i}_separat.jpg')), i))
#     except:
#         pass
#     try:
#         all_templates.append((read_image(os.path.join(folder_imagini_proprii, f'{i}_impreuna.jpg')), i))
#     except:
#         pass

# for template, val in all_templates:
#     template = template[10:-10, 10:-5]
#     centreaza_template(template, val)